In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from parseq.datasets import CFQDatasetLoader
from parseq.scripts_resplit.resplit_cfq import FrequencyDistribution, DivergenceComputer
import random, json
from typing import List
from parseq.grammar import taglisp_to_tree
from nltk import Tree
from tqdm import tqdm

In [ ]:
ds = CFQDatasetLoader().load(split="mcd1/modent", keepids=True)


In [ ]:
xs = ds.examples
random.shuffle(xs)

In [ ]:
def compute_example_overlap(comps, dist):   # how many of the compounds have already been observed in the given dist
    overlap = 0
    unoverlap = 0
    total = 0
    for comp in comps:
        if comp in dist and dist[comp] > 0:
            overlap += 1
        else:
            unoverlap += 1
        total += 1
    return overlap / total, unoverlap / total


def compute_example_stats(subsets, cds, dc=None):
    print("Computing stats for all examples")
    stats = {}
    cds = {k: v.tofreqs() for k, v in cds.items()}
    for subset in subsets:
        for example in tqdm(subset):
            eid = example[0]
            ecomp = dc.extract_compound_dist(example[2]).tofreqs()
            # print(len(ecomp))
            stats[eid] = (
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.9),
            )
    return stats


def make_mcd_splits(xs:List, sizes=(0.6, 0.2, 0.2), swapsize=10, iters=10):
    assert len(sizes) in (3,)
    # make random assignments and compute stats
    print("making random assigments")
    xs = [(x[0], x[1], taglisp_to_tree(x[2]) if not isinstance(x[2], Tree) else x[2]) for x in xs]
    random.shuffle(xs)
    subsets = []
    prev = 0
    for i, s in enumerate(sizes):
        if i < len(sizes):
            c = int(round(s * len(xs))) + prev
            subsets.append(xs[prev:c])
            prev = c
        else:
            subsets.append(xs[prev:])
            
    print("Computing initial compound distributions and their divergences")
    cds = {}
    dc = DivergenceComputer()
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)
    
    divs = dc._compute_compound_divergences(cds)
    print("Initial compound divergences")
    print(json.dumps(divs, indent=3))
    
    """
    print(subsets[0][0])
    comps = dc.extract_compounds(taglisp_to_tree(subsets[0][0][2]))
    print(compute_example_overlap(comps, cds["2"]))
    compdist = dc.extract_compound_dist(taglisp_to_tree(subsets[0][0][2]))
    print(FrequencyDistribution.compute_chernoff_coeff(cds["2"], compdist, alpha=0.1))
    """
    
    print("Iterating")
    # in every iteration, we find for each split the examples that would improve the divergence most and perform one swap
    # remove from train set all compounds that are present in test-ood and ood2
    # remove from ood2 all compounds that are present in test-ood
    # ==> test-ood would contain entirely novel compounds that have not been trained on and have not been validated with
    while iters > 0:
        stats = compute_example_stats(subsets, cds, dc=dc)   # initial stats
        
        # train --> test-ood
        examples = subsets[0]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintotest, traintotrain = scored[:swapsize], scored[swapsize:]
        
        # train -> ood2
        examples = traintotrain
        scored = [(i, stats[examples[i][0]][1]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintoood2, traintotrain = scored[:swapsize], scored[swapsize:]
        
        # ood2 -> test-ood
        examples = subsets[1]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totest, ood2toood2 = scored[:swapsize], scored[swapsize:]
        
        # ood2 -> train
        examples = ood2toood2
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totrain, ood2toood2 = scored[:swapsize], scored[swapsize:]
        
        # test-ood --> train
        examples = subsets[2]
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtotrain, testtotest = scored[:swapsize], scored[swapsize:]
        
        # test-ood --> ood2
        examples = testtotest
        scored = [(i, stats[examples[i][0]][4]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtoood2, testtotest = scored[:swapsize], scored[swapsize:]
        
        subsets = [traintotrain + ood2totrain + testtotrain, 
                   ood2toood2 + traintoood2 + testtoood2,
                   testtotest + traintotest + ood2totest]
        
        
        # updating compound distribution and computing divergences
        print("updating compound distributions and computing divergences")
        cds = {}
        for i, subset in enumerate(subsets):
            cds[str(i)] = dc.compute_compound_distribution(subset)
        divs = dc._compute_compound_divergences(cds)
        print(json.dumps(divs, indent=3))
        
        iters -= 1
    
            
    return cds

In [ ]:
dss = make_mcd_splits(xs[:1000])

In [ ]:
### len(dss[0]), len(dss[1]), len(dss[2])

In [ ]:
dss[0]._counts

In [ ]:
0.00001 ** 0.01

In [3]:
0.00001 ** 0.01

CFQDatasetLoader: make data
CFQDatasetLoader: make data in 0.0 sec
loading split 'mcd1'
splitting off a random 10% of 'train' for 'iidvalid' using seed 42
doing 'train'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86169/86169 [00:22<00:00, 3887.74it/s]


doing 'test'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11968/11968 [00:03<00:00, 3327.70it/s]


doing 'iidvalid'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9574/9574 [00:02<00:00, 3951.28it/s]


doing 'oodvalid'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11968/11968 [00:03<00:00, 3317.26it/s]


In [4]:
xs = ds.examples
random.shuffle(xs)

In [12]:
def compute_example_overlap(comps, dist):   # how many of the compounds have already been observed in the given dist
    overlap = 0
    unoverlap = 0
    total = 0
    for comp in comps:
        if comp in dist and dist[comp] > 0:
            overlap += 1
        else:
            unoverlap += 1
        total += 1
    return overlap / total, unoverlap / total


def compute_example_stats(subsets, cds, dc=None):
    print("Computing stats for all examples")
    stats = {}
    cds = {k: v.tofreqs() for k, v in cds.items()}
    for subset in subsets:
        for example in tqdm(subset):
            eid = example[0]
            ecomp = dc.extract_compound_dist(example[2]).tofreqs()
            # print(len(ecomp))
            stats[eid] = (
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.01),
                FrequencyDistribution.compute_chernoff_coeff(cds["0"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["1"], ecomp, alpha=0.9),
                FrequencyDistribution.compute_chernoff_coeff(cds["2"], ecomp, alpha=0.9),
            )
    return stats


def make_mcd_splits(xs:List, sizes=(0.6, 0.2, 0.2), swapsize=10, iters=10):
    assert len(sizes) in (3,)
    # make random assignments and compute stats
    print("making random assigments")
    xs = [(x[0], x[1], taglisp_to_tree(x[2]) if not isinstance(x[2], Tree) else x[2]) for x in xs]
    random.shuffle(xs)
    subsets = []
    prev = 0
    for i, s in enumerate(sizes):
        if i < len(sizes):
            c = int(round(s * len(xs))) + prev
            subsets.append(xs[prev:c])
            prev = c
        else:
            subsets.append(xs[prev:])
            
    print("Computing initial compound distributions and their divergences")
    cds = {}
    dc = DivergenceComputer()
    for i, subset in enumerate(subsets):
        cds[str(i)] = dc.compute_compound_distribution(subset)
    
    divs = dc._compute_compound_divergences(cds)
    print("Initial compound divergences")
    print(json.dumps(divs, indent=3))
    
    """
    print(subsets[0][0])
    comps = dc.extract_compounds(taglisp_to_tree(subsets[0][0][2]))
    print(compute_example_overlap(comps, cds["2"]))
    compdist = dc.extract_compound_dist(taglisp_to_tree(subsets[0][0][2]))
    print(FrequencyDistribution.compute_chernoff_coeff(cds["2"], compdist, alpha=0.1))
    """
    
    print("Iterating")
    # in every iteration, we find for each split the examples that would improve the divergence most and perform one swap
    # remove from train set all compounds that are present in test-ood and ood2
    # remove from ood2 all compounds that are present in test-ood
    # ==> test-ood would contain entirely novel compounds that have not been trained on and have not been validated with
    while iters > 0:
        stats = compute_example_stats(subsets, cds, dc=dc)   # initial stats
        
        # train --> test-ood
        examples = subsets[0]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintotest, traintotrain = scored[:swapsize], scored[swapsize:]
        
        # train -> ood2
        examples = traintotrain
        scored = [(i, stats[examples[i][0]][1]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        traintoood2, traintotrain = scored[:swapsize], scored[swapsize:]
        
        # ood2 -> test-ood
        examples = subsets[1]
        scored = [(i, stats[examples[i][0]][2]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totest, ood2toood2 = scored[:swapsize], scored[swapsize:]
        
        # ood2 -> train
        examples = ood2toood2
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        ood2totrain, ood2toood2 = scored[:swapsize], scored[swapsize:]
        
        # test-ood --> train
        examples = subsets[2]
        scored = [(i, stats[examples[i][0]][3]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtotrain, testtotest = scored[:swapsize], scored[swapsize:]
        
        # test-ood --> ood2
        examples = testtotest
        scored = [(i, stats[examples[i][0]][4]) for i in range(len(examples))]
        scored = sorted(scored, key=lambda x: x[1], reverse=True)
        scored = [examples[i] for i, score in scored]
        testtoood2, testtotest = scored[:swapsize], scored[swapsize:]
        
        subsets = [traintotrain + ood2totrain + testtotrain, 
                   ood2toood2 + traintoood2 + testtoood2,
                   testtotest + traintotest + ood2totest]
        
        
        # updating compound distribution and computing divergences
        print("updating compound distributions and computing divergences")
        cds = {}
        for i, subset in enumerate(subsets):
            cds[str(i)] = dc.compute_compound_distribution(subset)
        divs = dc._compute_compound_divergences(cds)
        print(json.dumps(divs, indent=3))
        
        iters -= 1
    
            
    return cds

In [13]:
dss = make_mcd_splits(xs[:1000])

making random assigments
Computing initial compound distributions and their divergences


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1129.10it/s]


Initial compound divergences
{
   "0-0": -3.9124259387790516e-13,
   "0-1": 0.5250916339795171,
   "0-2": 0.555126373303761,
   "1-0": 0.6578991159015115,
   "1-1": 9.481304630298837e-14,
   "1-2": 0.6901829090683067,
   "2-0": 0.6580363095261349,
   "2-1": 0.665918151138407,
   "2-2": -2.327027459614328e-13
}
Iterating
Computing stats for all examples


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 921.46it/s]


updating compound distributions and computing divergences


...

{
   "0-0": 2.667865928174251e-13,
   "0-1": 0.6575852822672847,
   "0-2": 0.5375946508122367,
   "1-0": 0.8567379827178419,
   "1-1": 1.2445600106048005e-13,
   "1-2": 0.8101163716336874,
   "2-0": 0.7734075170099721,
   "2-1": 0.7866545206448482,
   "2-2": 5.81756864903582e-14
}
Computing stats for all examples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1317.98it/s]


updating compound distributions and computing divergences


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1673.69it/s]


{
   "0-0": 1.9517720772910252e-13,
   "0-1": 0.6721809373543337,
   "0-2": 0.5430507244363834,
   "1-0": 0.8636224263672824,
   "1-1": -8.859579736508749e-14,
   "1-2": 0.8013676581095405,
   "2-0": 0.7683812155658005,
   "2-1": 0.7950097182180946,
   "2-2": 8.79296635503124e-14
}


In [9]:
### len(dss[0]), len(dss[1]), len(dss[2])

In [10]:
dss[0]._counts

KeyError: 0

In [193]:
0.00001 ** 0.01

0.8912509381337456